# Install Dependencies

In [ ]:
!pip install gym_anytrading

### Imports

In [ ]:
import numpy as np
import pandas as pd

import gym
import gym_anytrading

from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv

import matplotlib.pyplot as plt

# Load and Review Dataset

In [ ]:
df = gym_anytrading.datasets.STOCKS_GOOGL.copy()

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
len(df)

# Register the stock trading environment

In [ ]:
window_size = 10
start_index = window_size
end_index = len(df)

env_maker = lambda: gym.make(
    'stocks-v0',
    df = df,
    window_size = window_size,
    frame_bound = (start_index, end_index)
)

env = DummyVecEnv([env_maker])

The stock trading environment is a good example of a custom environment. A custom environment follows the OpenAI gym interface and at minimum needs to provide these methods:

- `__init__()`: initializes the instance of the environment. The most important thing this method does is define the action space and the observation space.
- `reset()`: resets the state of the environment and returns the initial observation.
- `step(action)`: executes the specified action in the environment, returning the observation that results along with the immediate reward, a flag indicating if the episode is ended and additional information specific to the scenario.

Let's explore some fo the highlights of the stocks-v0 environment. Make sure to follow the inline links to see the code being described in the source repo.

## The base environment class

The `gym_anytrading` library provides the `TradingEnv` base class that represents the general case for any form of trading environment. 

It takes the simplest approach of defining the [Actions](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/trading_env.py#L9) that an agent can select from as simply either `Sell` or `Buy`. 





## Environment Initialization

In the `__init__` method of the `TradingEnv` class it defines the [action space](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/trading_env.py#L36) as a discrete action space (e.g., the values are either 0 for sell or 1 for buy). It also defines the [observation space](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/trading_env.py#L37) as continous, as a `Box` (e.g., the real values range between negative infinity and positive infinity).

In other words, in this environment the agent has discrete options to buy or sell. The observation the environment returns after performing the action reflect the next price of the asset at the next tick in time of the dataset.  

## Environment Reset

The [reset](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/trading_env.py#L58) method does what you would expect, it resets the state of the environment (e.g., set the accumulated reward to 0, set the tick counter back to the start) and returns an observation that is basically the first price from the history of price data of the asset being traded.



## Environment Step

The [step](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/trading_env.py#L71) method is where the action the agent selected affects the state of the environment. In the TradingBase class this involves addressing the following:

- _Are we done?_ Compute if the action resulted in the episode being over by [setting the done flag](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/trading_env.py#L75). In this case, the episode is over when we have reached the last tick in the range of ticks provided. 
- _What reward did the agent earn?_ The [immediate reward that results](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/trading_env.py#L78) from the taking the action the agent requested is calculated. In the base class, this is just a stub function that is specialized by derived classes, such as that stock trading class we will cover shortly. 
- _Update environment internal state._ In this case the environment [updates its running total of the rewards earned](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/trading_env.py#L79), the [profit so far](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/trading_env.py#L81), and if a trade took place [toggling its position from a Short to a Long](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/trading_env.py#L89) or vice-versa.
- Return the [observation, immediate reward, done flag and any additional metadata](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/trading_env.py#L101) that the environment can communicate to the agent.

# The stocks-v0 environment

The `stocks-v0` environment we loaded previously is represented by the [StocksEnv class](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/stocks_env.py#L6) which derives from the TradingEnv class.

In the [init of the environment](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/stocks_env.py#L8), it configures a default fee amount for bid and ask trades. These fees are expressed in terms of units (e.g., a bid fee is 1% of one share).

The [immediate reward is calculated](https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/envs/stocks_env.py#L41) if a trade took place. It earns a reward that is the difference in price between the current price and when the agent last made a trade. The reward provided to the agent does not incorporate buy or sell fees.

The reward is calculated as follows:
- Positive reward: An agent earns a positive reward when it sells a stock at a higher price than what it had bought it at previously. 
- Negative reward: It might earn a negative reward if it sold below the price at which it bought. 
- Zero reward: If the agent did not make a trade, or if the trade placed was to buy the stock. In other words, the agent only gets a reward when it sells something it had previously bought.

# Train the trading agent

In [ ]:
policy_kwargs = dict(net_arch=[dict(vf=[128, 128, 128], pi=[64, 64])])
model = A2C('MlpPolicy', env, verbose=1, policy_kwargs=policy_kwargs)
model.learn(total_timesteps=1000)

# Evaluate the agent

In [ ]:
env = env_maker()
observation = env.reset()

while True:
    observation = observation[np.newaxis, ...]
    action, _states = model.predict(observation)
    observation, reward, done, info = env.step(action)

    #env.render()
    if done:
        print("info:", info)
        break

## Plot Results

In [ ]:
plt.figure(figsize=(16, 6))
env.render_all()
plt.show()

## Examine the quantative results

In [ ]:
!pip install quantstats --upgrade --no-cache-dir

In [ ]:
import quantstats as qs
qs.extend_pandas()

net_worth = pd.Series(env.history['total_profit'], index=df.index[start_index+1:end_index])
returns = net_worth.pct_change().iloc[1:]

qs.reports.full(returns)
qs.reports.html(returns, output='a2c_quantstats.html')

So it's not a great agent (there are lot of things that you could do to improve its performance), but that was not the point- the goal was to learn how to build an environment for a novel scenario.